# 🔄 OpenCV Geometric Transformations

## 學習目標
- 掌握影像的縮放（Scaling）操作
- 理解影像的旋轉（Rotation）技術
- 學習仿射變換（Affine Transformation）
- 實作透視變換（Perspective Transformation）
- 應用影像翻轉與平移

---

## 📚 1. 幾何變換基礎概念

### 什麼是幾何變換？

**幾何變換**是改變影像的幾何結構，包括位置、方向、大小和形狀的操作。

### 主要類型
- 🔍 **縮放（Scaling）**: 改變影像大小
- 🔄 **旋轉（Rotation）**: 旋轉影像角度
- ↔️ **翻轉（Flipping）**: 水平或垂直翻轉
- 📐 **仿射變換（Affine）**: 保持平行線的變換
- 🎭 **透視變換（Perspective）**: 模擬3D視角變化

In [ ]:
# 導入必要的庫
import cv2
import numpy as np
import matplotlib.pyplot as plt
from utils.image_utils import load_image, resize_image
from utils.visualization import display_multiple_images

# 設置 matplotlib 顯示中文
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 環境準備完成！")
print(f"OpenCV 版本: {cv2.__version__}")

In [ ]:
# 載入測試影像
image = load_image('../assets/images/basic/lenaColor.png')

if image is not None:
    print(f"✅ 影像載入成功！")
    print(f"📐 原始尺寸: {image.shape[1]} x {image.shape[0]}")
    
    # 顯示原始影像
    plt.figure(figsize=(6, 6))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title('原始影像')
    plt.axis('off')
    plt.show()
else:
    print("❌ 影像載入失敗！")

## 🔍 2. 影像縮放（Scaling）

### 縮放方法
- `cv2.INTER_NEAREST`: 最近鄰插值（速度最快，品質最低）
- `cv2.INTER_LINEAR`: 雙線性插值（預設，速度與品質平衡）
- `cv2.INTER_CUBIC`: 雙三次插值（品質較高）
- `cv2.INTER_LANCZOS4`: Lanczos插值（品質最高，速度較慢）

In [ ]:
# 方法1: 指定新尺寸
if image is not None:
    # 縮小為原來的 50%
    new_size = (256, 256)
    resized_small = cv2.resize(image, new_size)
    
    # 放大為原來的 150%
    new_size_large = (768, 768)
    resized_large = cv2.resize(image, new_size_large)
    
    # 顯示結果
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(cv2.cvtColor(resized_small, cv2.COLOR_BGR2RGB))
    axes[0].set_title(f'縮小 (256x256)')
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f'原始 ({image.shape[1]}x{image.shape[0]})')
    axes[1].axis('off')
    
    axes[2].imshow(cv2.cvtColor(resized_large, cv2.COLOR_BGR2RGB))
    axes[2].set_title(f'放大 (768x768)')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 方法2: 使用縮放係數
if image is not None:
    # 使用 fx, fy 參數（縮放倍數）
    scaled_down = cv2.resize(image, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
    scaled_up = cv2.resize(image, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
    
    print(f"縮小後尺寸: {scaled_down.shape}")
    print(f"放大後尺寸: {scaled_up.shape}")

In [ ]:
# 比較不同插值方法的效果
if image is not None:
    # 先縮小再放大，觀察插值差異
    small = cv2.resize(image, (128, 128))
    
    nearest = cv2.resize(small, (512, 512), interpolation=cv2.INTER_NEAREST)
    linear = cv2.resize(small, (512, 512), interpolation=cv2.INTER_LINEAR)
    cubic = cv2.resize(small, (512, 512), interpolation=cv2.INTER_CUBIC)
    lanczos = cv2.resize(small, (512, 512), interpolation=cv2.INTER_LANCZOS4)
    
    # 顯示對比
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    
    axes[0, 0].imshow(cv2.cvtColor(nearest, cv2.COLOR_BGR2RGB))
    axes[0, 0].set_title('INTER_NEAREST (最快)')
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(cv2.cvtColor(linear, cv2.COLOR_BGR2RGB))
    axes[0, 1].set_title('INTER_LINEAR (預設)')
    axes[0, 1].axis('off')
    
    axes[1, 0].imshow(cv2.cvtColor(cubic, cv2.COLOR_BGR2RGB))
    axes[1, 0].set_title('INTER_CUBIC (品質較高)')
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(cv2.cvtColor(lanczos, cv2.COLOR_BGR2RGB))
    axes[1, 1].set_title('INTER_LANCZOS4 (品質最高)')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()

## 🔄 3. 影像旋轉（Rotation）

### 旋轉方法
1. 使用 `cv2.getRotationMatrix2D()` 獲取旋轉矩陣
2. 使用 `cv2.warpAffine()` 應用變換

In [ ]:
# 基本旋轉
if image is not None:
    height, width = image.shape[:2]
    center = (width // 2, height // 2)
    
    # 獲取旋轉矩陣（中心點，角度，縮放倍數）
    angles = [45, 90, 135, 180]
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    
    for i, angle in enumerate(angles):
        # 獲取旋轉矩陣
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        
        # 執行旋轉
        rotated = cv2.warpAffine(image, rotation_matrix, (width, height))
        
        # 顯示結果
        row, col = i // 2, i % 2
        axes[row, col].imshow(cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB))
        axes[row, col].set_title(f'旋轉 {angle}°')
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# 旋轉並調整畫布大小（避免裁切）
if image is not None:
    height, width = image.shape[:2]
    center = (width // 2, height // 2)
    angle = 45
    
    # 計算旋轉後的新邊界
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    
    # 計算新的畫布尺寸
    cos = np.abs(rotation_matrix[0, 0])
    sin = np.abs(rotation_matrix[0, 1])
    new_width = int((height * sin) + (width * cos))
    new_height = int((height * cos) + (width * sin))
    
    # 調整旋轉矩陣的平移部分
    rotation_matrix[0, 2] += (new_width / 2) - center[0]
    rotation_matrix[1, 2] += (new_height / 2) - center[1]
    
    # 執行旋轉（無裁切）
    rotated_normal = cv2.warpAffine(image, cv2.getRotationMatrix2D(center, angle, 1.0), (width, height))
    rotated_expanded = cv2.warpAffine(image, rotation_matrix, (new_width, new_height))
    
    # 顯示對比
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axes[0].set_title('原始影像')
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(rotated_normal, cv2.COLOR_BGR2RGB))
    axes[1].set_title('旋轉45° (有裁切)')
    axes[1].axis('off')
    
    axes[2].imshow(cv2.cvtColor(rotated_expanded, cv2.COLOR_BGR2RGB))
    axes[2].set_title('旋轉45° (完整顯示)')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()

## ↔️ 4. 影像翻轉（Flipping）

In [ ]:
# 翻轉操作
if image is not None:
    # flipCode = 0: 垂直翻轉
    flipped_vertical = cv2.flip(image, 0)
    
    # flipCode = 1: 水平翻轉
    flipped_horizontal = cv2.flip(image, 1)
    
    # flipCode = -1: 水平+垂直翻轉（旋轉180度）
    flipped_both = cv2.flip(image, -1)
    
    # 顯示結果
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    
    axes[0, 0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axes[0, 0].set_title('原始影像')
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(cv2.cvtColor(flipped_horizontal, cv2.COLOR_BGR2RGB))
    axes[0, 1].set_title('水平翻轉 (flipCode=1)')
    axes[0, 1].axis('off')
    
    axes[1, 0].imshow(cv2.cvtColor(flipped_vertical, cv2.COLOR_BGR2RGB))
    axes[1, 0].set_title('垂直翻轉 (flipCode=0)')
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(cv2.cvtColor(flipped_both, cv2.COLOR_BGR2RGB))
    axes[1, 1].set_title('雙向翻轉 (flipCode=-1)')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()

## 📐 5. 仿射變換（Affine Transformation）

仿射變換保持平行線的平行關係，需要3個點來定義變換矩陣。

### 應用場景
- 傾斜校正
- 影像對齊
- 變形效果

In [ ]:
# 仿射變換範例
if image is not None:
    height, width = image.shape[:2]
    
    # 定義3個源點和目標點
    src_points = np.float32([
        [50, 50],
        [width - 50, 50],
        [50, height - 50]
    ])
    
    dst_points = np.float32([
        [10, 100],
        [width - 50, 50],
        [100, height - 50]
    ])
    
    # 獲取仿射變換矩陣
    affine_matrix = cv2.getAffineTransform(src_points, dst_points)
    
    # 執行仿射變換
    affine_result = cv2.warpAffine(image, affine_matrix, (width, height))
    
    # 在原圖上標記源點
    image_with_points = image.copy()
    for point in src_points:
        cv2.circle(image_with_points, tuple(point.astype(int)), 5, (0, 255, 0), -1)
    
    # 在變換後的圖上標記目標點
    result_with_points = affine_result.copy()
    for point in dst_points:
        cv2.circle(result_with_points, tuple(point.astype(int)), 5, (0, 0, 255), -1)
    
    # 顯示結果
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    axes[0].imshow(cv2.cvtColor(image_with_points, cv2.COLOR_BGR2RGB))
    axes[0].set_title('原始影像 (綠點=源點)')
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(result_with_points, cv2.COLOR_BGR2RGB))
    axes[1].set_title('仿射變換後 (紅點=目標點)')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("仿射變換矩陣:")
    print(affine_matrix)

## 🎭 6. 透視變換（Perspective Transformation）

透視變換可以模擬3D視角變化，需要4個點來定義變換矩陣。

### 應用場景
- 文件掃描校正
- 車牌識別
- 鳥瞰圖轉換

In [ ]:
# 透視變換範例 - 模擬文件掃描
if image is not None:
    height, width = image.shape[:2]
    
    # 定義4個源點（傾斜的四邊形）
    src_points = np.float32([
        [100, 100],      # 左上
        [width - 50, 80],   # 右上
        [50, height - 80],  # 左下
        [width - 100, height - 100]  # 右下
    ])
    
    # 定義4個目標點（正矩形）
    dst_points = np.float32([
        [0, 0],
        [width - 1, 0],
        [0, height - 1],
        [width - 1, height - 1]
    ])
    
    # 獲取透視變換矩陣
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    
    # 執行透視變換
    perspective_result = cv2.warpPerspective(image, perspective_matrix, (width, height))
    
    # 在原圖上繪製源區域
    image_with_region = image.copy()
    cv2.polylines(image_with_region, [src_points.astype(np.int32)], True, (0, 255, 0), 3)
    for i, point in enumerate(src_points):
        cv2.circle(image_with_region, tuple(point.astype(int)), 8, (0, 255, 0), -1)
        cv2.putText(image_with_region, str(i+1), tuple(point.astype(int) - 20), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # 顯示結果
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    axes[0].imshow(cv2.cvtColor(image_with_region, cv2.COLOR_BGR2RGB))
    axes[0].set_title('原始影像 (綠框=要校正的區域)')
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(perspective_result, cv2.COLOR_BGR2RGB))
    axes[1].set_title('透視變換後 (已校正)')
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("透視變換矩陣:")
    print(perspective_matrix)

## 📋 7. 實際應用範例：文件掃描校正

In [ ]:
# 完整的文件掃描校正流程
def document_scanner(image):
    """
    模擬文件掃描校正流程
    """
    # 1. 轉換為灰階
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 2. 邊緣檢測
    edges = cv2.Canny(gray, 50, 150)
    
    # 3. 尋找輪廓
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 4. 找出最大的四邊形輪廓
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    document_contour = None
    for contour in contours[:5]:  # 檢查前5個最大輪廓
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)
        if len(approx) == 4:
            document_contour = approx
            break
    
    if document_contour is None:
        print("❌ 未找到文件邊界")
        return image
    
    # 5. 排序四個角點
    points = document_contour.reshape(4, 2)
    rect = np.zeros((4, 2), dtype="float32")
    
    s = points.sum(axis=1)
    rect[0] = points[np.argmin(s)]  # 左上（x+y最小）
    rect[2] = points[np.argmax(s)]  # 右下（x+y最大）
    
    diff = np.diff(points, axis=1)
    rect[1] = points[np.argmin(diff)]  # 右上（x-y最小）
    rect[3] = points[np.argmax(diff)]  # 左下（x-y最大）
    
    # 6. 計算目標尺寸
    (tl, tr, br, bl) = rect
    width_a = np.linalg.norm(br - bl)
    width_b = np.linalg.norm(tr - tl)
    max_width = max(int(width_a), int(width_b))
    
    height_a = np.linalg.norm(tr - br)
    height_b = np.linalg.norm(tl - bl)
    max_height = max(int(height_a), int(height_b))
    
    # 7. 透視變換
    dst = np.array([
        [0, 0],
        [max_width - 1, 0],
        [max_width - 1, max_height - 1],
        [0, max_height - 1]
    ], dtype="float32")
    
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (max_width, max_height))
    
    return warped, document_contour

# 測試文件掃描
if image is not None:
    # 創建一個傾斜的測試影像
    h, w = image.shape[:2]
    src_pts = np.float32([[0, 0], [w, 0], [w, h], [0, h]])
    dst_pts = np.float32([[50, 100], [w-50, 50], [w-100, h-50], [100, h-100]])
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    tilted = cv2.warpPerspective(image, M, (w, h))
    
    # 執行掃描校正
    result = document_scanner(tilted)
    
    if isinstance(result, tuple):
        scanned, contour = result
        
        # 在傾斜影像上繪製檢測到的輪廓
        tilted_with_contour = tilted.copy()
        cv2.drawContours(tilted_with_contour, [contour], -1, (0, 255, 0), 3)
        
        # 顯示結果
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        
        axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        axes[0].set_title('1. 原始影像')
        axes[0].axis('off')
        
        axes[1].imshow(cv2.cvtColor(tilted_with_contour, cv2.COLOR_BGR2RGB))
        axes[1].set_title('2. 傾斜影像（檢測邊界）')
        axes[1].axis('off')
        
        axes[2].imshow(cv2.cvtColor(scanned, cv2.COLOR_BGR2RGB))
        axes[2].set_title('3. 掃描校正結果')
        axes[2].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        print("✅ 文件掃描校正完成！")

## 🎯 8. 實用工具函數

In [ ]:
def rotate_bound(image, angle):
    """
    旋轉影像並自動調整畫布大小，避免裁切
    
    Args:
        image: 輸入影像
        angle: 旋轉角度（度）
    
    Returns:
        rotated: 旋轉後的影像
    """
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    
    # 獲取旋轉矩陣
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    
    # 計算新的邊界
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    
    # 調整旋轉矩陣
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
    
    return cv2.warpAffine(image, M, (nW, nH))

# 測試工具函數
if image is not None:
    rotated_30 = rotate_bound(image, 30)
    rotated_60 = rotate_bound(image, 60)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axes[0].set_title('原始')
    axes[0].axis('off')
    
    axes[1].imshow(cv2.cvtColor(rotated_30, cv2.COLOR_BGR2RGB))
    axes[1].set_title('旋轉30° (無裁切)')
    axes[1].axis('off')
    
    axes[2].imshow(cv2.cvtColor(rotated_60, cv2.COLOR_BGR2RGB))
    axes[2].set_title('旋轉60° (無裁切)')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()

## 🎯 9. 課程總結

### ✅ 本課程已學習：

1. **影像縮放（Scaling）**
   - 使用 `cv2.resize()` 改變影像大小
   - 不同插值方法的特點和應用場景

2. **影像旋轉（Rotation）**
   - 使用 `cv2.getRotationMatrix2D()` 和 `cv2.warpAffine()`
   - 避免旋轉裁切的技巧

3. **影像翻轉（Flipping）**
   - 水平、垂直、雙向翻轉

4. **仿射變換（Affine）**
   - 使用3個點定義變換
   - 保持平行線平行的特性

5. **透視變換（Perspective）**
   - 使用4個點定義變換
   - 模擬3D視角變化
   - 文件掃描校正應用

### 🚀 下一步：
- 影像算術運算
- 影像混合與融合
- 色彩空間深入探討

## 📝 練習題

### 基礎練習
1. 將影像縮放為原來的1/4大小，並比較不同插值方法的效果
2. 實現一個函數，將影像旋轉任意角度並保持完整顯示
3. 創建一個鏡像效果：左右兩側分別顯示原圖和水平翻轉圖

### 進階練習
1. 實現一個簡單的「照片拼貼」功能，將4張圖片排列成2x2網格
2. 實現一個傾斜校正工具，自動檢測並校正傾斜的文件影像
3. 創建一個「魚眼效果」濾鏡

### 挑戰題
1. 實現一個全景圖拼接程序（使用仿射或透視變換）
2. 創建一個車牌識別預處理程序（透視校正+旋轉校正）
3. 實現一個「鳥瞰圖」轉換工具（適用於停車場監控）

---

*🎯 OpenCV Computer Vision Toolkit - 幾何變換完成！*